# K-means Method

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.contrib.factorization import KMeans
# Ignore all GPUs, tf random forest does not benefit from it.
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
features = np.loadtxt("features.txt", delimiter=",")
labels = np.loadtxt("labels.txt", delimiter=",")
x_all = features
#y_all = labels
y_all = np.array([labels, -(labels-1)]).T 
# normalize
x_all = (x_all - x_all.min(0)) / x_all.ptp(0)
#Split the data in train & test
#Y_reshape = data[:,2].reshape(data[:,2].shape[0], 1)
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all)

In [3]:
# Parameters
num_steps = 100 # Total steps to train
batch_size = 1024 # The number of samples per batch
k = 30 # The number of clusters
num_classes = 2 # The 10 digits
num_features = 244 # Each image is 244 bands

# Input data
X = tf.placeholder(tf.float32, shape=[None, num_features])
# Labels (for assigning a label to a centroid and testing)
Y = tf.placeholder(tf.float32, shape=[None, num_classes])

# K-Means Parameters
kmeans = KMeans(inputs=X, num_clusters=k, distance_metric='cosine',
                use_mini_batch=True)


# Build KMeans graph
(all_scores, cluster_idx, scores, cluster_centers_initialized,init_op,train_op) = kmeans.training_graph()
cluster_idx = cluster_idx[0] 
#all_scores, cluster_idx, scores, cluster_centers_initialized, cluster_centers_vars,init_op,train_op) = kmeans.training_graph()
#cluster_idx = cluster_idx[0] # fix for cluster_idx being a tuple
 
avg_distance = tf.reduce_mean(scores)

# Initialize the variables (i.e. assign their default value)
init_vars = tf.global_variables_initializer()

# Start TensorFlow session
sess = tf.Session()

# Run the initializer
sess.run(init_vars, feed_dict={X: x_train})
sess.run(init_op, feed_dict={X: x_train})

# Training
for i in range(1, num_steps + 1):
    _, d, idx = sess.run([train_op, avg_distance, cluster_idx],
                         feed_dict={X: x_train})
    if i % 10 == 0 or i == 1:
        print("Step %i, Avg Distance: %f" % (i, d))

# Assign a label to each centroid
# Count total number of labels per centroid, using the label of each training
# sample to their closest centroid (given by 'idx')
counts = np.zeros(shape=(k, num_classes))
for i in range(len(idx)):
    counts[idx[i]] += y_train[i]
# Assign the most frequent label to the centroid
labels_map = [np.argmax(c) for c in counts]
labels_map = tf.convert_to_tensor(labels_map)

# Evaluation ops
# Lookup: centroid_id -> label
cluster_label = tf.nn.embedding_lookup(labels_map, cluster_idx)
# Compute accuracy
correct_prediction = tf.equal(cluster_label, tf.cast(tf.argmax(Y, 1), tf.int32))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Test Model
#test_x, test_y = mnist.test.images, mnist.test.labels
print("Train Accuracy:", sess.run(accuracy_op, feed_dict={X: x_train, Y: y_train}))
print("Test Accuracy:", sess.run(accuracy_op, feed_dict={X: x_test, Y: y_test}))

Instructions for updating:
dim is deprecated, use axis instead
Step 1, Avg Distance: 0.007974
Step 10, Avg Distance: 0.004258
Step 20, Avg Distance: 0.004147
Step 30, Avg Distance: 0.004103
Step 40, Avg Distance: 0.004079
Step 50, Avg Distance: 0.004062
Step 60, Avg Distance: 0.004050
Step 70, Avg Distance: 0.004041
Step 80, Avg Distance: 0.004033
Step 90, Avg Distance: 0.004026
Step 100, Avg Distance: 0.004020
Train Accuracy: 0.8514079
Test Accuracy: 0.840301
